In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
from ds.data.sets import load_sets_v2
import joblib
import pandas as pd
import tensorflow as tf
from sklearn.impute import SimpleImputer
import numpy as np

In [55]:
tf.keras.backend.clear_session()
tf.keras.utils.set_random_seed(55)

In [56]:
datasets = joblib.load("../data/processed/datasetsjmone")

In [57]:
drops = [
    "review_overall",
    "beer_name",
]
for data in datasets:
    if isinstance(data, pd.DataFrame):
        for column in drops:
            if column in data.columns:
                data.drop(columns=column, inplace=True)

In [58]:
x_train, y_train, x_test, y_test, x_validation, y_validation = datasets

In [59]:
x_train

,brewery_name,review_aroma,review_appearance,review_palate,review_taste,beer_abv
590715,goose island beer co,5.0,4.5,4.0,5.0,13.00
1514329,hinterland brewery restaurant,4.0,4.0,3.5,3.5,6.85
272592,flying dog brewery,3.5,3.0,3.5,3.5,5.50
894213,stone brewing co,4.0,4.5,4.0,4.0,7.20
174214,long trail brewing co,4.0,4.0,4.0,4.0,8.60
...,...,...,...,...,...,...
759715,widmer brothers brewing company,3.5,4.5,3.0,3.5,9.40
618842,harpoon brewery,3.5,3.5,3.5,3.5,4.30
948973,bells brewery inc,3.5,3.5,3.5,3.5,5.20
678833,moorhouses brewery burnley ltd,3.5,4.0,3.0,3.5,3.40


In [60]:
x_train = x_train.fillna(-9)
x_test = x_test.fillna(-9)

In [61]:
BATCH_SIZE = 10000

In [62]:
df = x_train


In [63]:
y = y_train
y_vocab = sorted(set(y))

In [64]:
y_tensor = tf.convert_to_tensor(y, dtype=tf.string)

In [65]:
class TargetDummy(tf.keras.Model):

  def __init__(self, vocab=None):
    super().__init__()
    self.vocab = vocab
    self.lookup = tf.keras.layers.StringLookup(vocabulary=self.vocab, output_mode='one_hot')
    if vocab is None:
      self._vocab = vocab
    else:
      self._vocab = self.lookup.get_vocabulary()
    self.invert = tf.keras.layers.StringLookup(vocabulary=self._vocab, invert=True)
    self.input_lyr = tf.keras.Input(shape=(), dtype=tf.string)
    self.inputs = self.input_lyr
  
  def vocabulary_size(self):
    return self.lookup.vocabulary_size()

  def adapt(self, y_train):
    self.lookup.adapt(y_train)
    self._vocab = self.lookup.get_vocabulary()
    self.invert.set_vocabulary(vocabulary=self._vocab)    

  def call(self, inputs, training=False, invert=False):
    if invert is True:
      x = tf.argmax(inputs, axis=-1)
      x = self.invert(x)
    else:
      x = self.lookup(inputs)
      self.invert.build(inputs)
    return x

In [66]:
# target_model = TargetDummy(y_vocab)
# look = target_model(y_tensor)
# back = target_model(look, invert=True)
# back

In [67]:
# target_model = TargetDummy()
# test_y_tens = y_tensor[:5]
# target_model.adapt(test_y_tens)
# look = target_model(test_y_tens)
# back = target_model(look, invert=True)
# back

In [68]:
target_lookup = TargetDummy(y_vocab)
target = target_lookup(y_tensor)

In [69]:
ds = tf.data.Dataset.from_tensor_slices((dict(df), target))
ds = ds.batch(BATCH_SIZE)


In [70]:
y_test_tensor = tf.convert_to_tensor(y_test)
y_test_encoded = target_lookup(y_test_tensor)
test = tf.data.Dataset.from_tensor_slices((dict(x_test), y_test_encoded))
test = test.batch(BATCH_SIZE)

In [71]:
# #compute imputer mean
# mask = tf.where(tf.math.is_nan(tensor) , 0. , tensor)
# mask_norm = tf.reduce_sum(tf.clip_by_value(mask, 0., 1.),axis=0)
# imp_mean = tf.math.divide(tf.reduce_sum(mask, axis=0), mask_norm)

# #transform
# tf.where(tf.math.is_nan(X) , imp_mean , X)

In [72]:
numeric_feature_names = [
    # "review_overall",
    "review_aroma",
    "review_appearance",
    "review_palate",
    "review_taste",
]
numeric_features = df[numeric_feature_names]

categorical_feature_names = [
    "brewery_name"
]

In [73]:
def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
      values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

In [74]:
inputs = {}
preprocessed = []

In [75]:
for name, column in x_train.items():
  if type(column[0]) == str:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)

In [76]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(numeric_features)))

In [77]:
numeric_inputs = {}
for name in numeric_feature_names:
  numeric_inputs[name]=inputs[name]

numeric_inputs = stack_dict(numeric_inputs)
numeric_normalized = normalizer(numeric_inputs)

preprocessed.append(numeric_normalized)

preprocessed

[<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'normalization')>]

In [78]:
for name in categorical_feature_names:
  vocab = sorted(set(df[name]))

  if type(vocab[0]) is str:
    lookup = tf.keras.layers.StringLookup(vocabulary=vocab, output_mode='one_hot')
  else:
    lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, output_mode='one_hot')

  x = inputs[name][:, tf.newaxis]
  x = lookup(x)
  preprocessed.append(x)

In [79]:
preprocesssed_result = tf.concat(preprocessed, axis=-1)
preprocessor = tf.keras.Model(inputs, preprocesssed_result)
tf.keras.utils.plot_model(preprocessor, rankdir="LR", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [80]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    self.dense1= tf.keras.layers.Dense(2048, activation='relu')
    self.dense2 = tf.keras.layers.Dense(1024, activation='relu')
    self.dense3 = tf.keras.layers.Dense(512, activation='relu')
    self.dense4 = tf.keras.layers.Dense(2048, activation='relu')
    self.dense5 = tf.keras.layers.Dense(1024, activation='relu')
    self.attn1 = tf.keras.layers.Attention()
    self.attn2 = tf.keras.layers.Attention()
    self.dropout1 = tf.keras.layers.Dropout(0.2)
    self.dropout2 = tf.keras.layers.Dropout(0.2)
    self.dropout2 = tf.keras.layers.Dropout(0.2)
    self.norm1 = tf.keras.layers.BatchNormalization()
    self.norm2 = tf.keras.layers.BatchNormalization()
    self.norm3 = tf.keras.layers.BatchNormalization()

    self.outputlyr = tf.keras.layers.Dense(target_lookup.vocabulary_size(), activation='softmax')
    self.flatten = tf.keras.layers.Flatten()
    self.mask = tf.keras.layers.Masking(mask_value=0.0)
    
  def call(self, inputs, training=False):
    # x = self.flatten(inputs)
    x = self.dense1(inputs)
    # x = self.attn1(([x, x]))
    x = self.dense2(x)
    x = self.norm1(x)
    if training:
      x = self.dropout1(x, training=training)
    x = self.dense3(x)
    x = self.norm2(x)
    if training:
      x = self.dropout2(x, training=training)
    x = self.dense4(x)
    # x = self.dense5(x)

    return self.outputlyr(x)

body = MyModel()

In [81]:
x = preprocessor(inputs)
x

<KerasTensor: shape=(None, 5862) dtype=float32 (created by layer 'model')>

In [82]:
result = body(x)
result

<KerasTensor: shape=(None, 105) dtype=float32 (created by layer 'my_model')>

In [83]:
model = tf.keras.Model(inputs, result)

model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])

In [84]:
history = model.fit(
    ds,
    epochs=200,
    batch_size=BATCH_SIZE,
    validation_data=test
    )

Epoch 1/200


2023-07-11 13:27:47.750093: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype double and shape [1361552]
	 [[{{node Placeholder/_4}}]]
2023-07-11 13:27:48.830982: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x53b77e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-11 13:27:48.831009: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2023-07-11 13:27:48.833741: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-11 13:27:48.929216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-0

 44/137 [========>.....................] - ETA: 9s - loss: 3.1152 - accuracy: 0.2256

KeyboardInterrupt: 

In [ ]:
from pathlib import Path

def get_version_num(path):
    return int(path.stem.split("_")[-1])

In [ ]:
base_path = Path("../models/gpu/")

model_name = "densebatchnorm"
existing_versions = base_path.glob(f"{model_name}*")
existing_versions = [get_version_num(model_path) for model_path in existing_versions]
if len(existing_versions) == 0:
    version_num = 1
else:
    version_num = max(existing_versions) + 1

this_model = f"{model_name}_{version_num}"

save_path = base_path.joinpath(this_model)

target_lookup_path = base_path.joinpath(f"{this_model}_vocabmodel")

model.save(save_path)



NameError: name 'model' is not defined

INFO:tensorflow:Assets written to: ./testsave/assets


In [ ]:
test_reverse = t_model(t_var)
test_reverse

<tf.Tensor: shape=(20, 105), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [ ]:
t_model(test_reverse, invert=True)

TypeError: call() got an unexpected keyword argument 'invert'